# build_compilation notebook

This notebook iterates through each paleomagnetic record (datasheet) of the vgp database, extracts data which meet user-specified criteria, and appends them to a new dataframe for later processing (to generate an APWP).

In [1]:
import os
import re
import numpy as np
import pandas as pd
from pmagpy import ipmag, pmag
import scripts.auxiliar as aux

pd.set_option('display.max_columns', None)

Set the directory from which we will pull the datasheets.

In [2]:
data_path_vgp = os.getcwd() + '/vgp_database'
files_names = aux.get_files_in_directory(data_path_vgp)
csv_file_names = [os.path.splitext(os.path.basename(open(file,'r').name))[0] for file in files_names if file.endswith('.csv')] #consider just *csv files
paths = [file for file in files_names if file.endswith('.csv')] 
files = pd.DataFrame({'path': paths, 'name': csv_file_names})

### Set data inclusion criteria
Specify the inclusion criteria to be used in the data-selection. If author_selection is set=1, all other criteria will be ignored. Setting values other than 'None' for the remaining criteria allow them to be homogenized across studies.

In [3]:
incl_criteria = {
    'author_selection': 1,     # 1 (yes) or 0 (no); if 1, all other criteria will be ignored
    'undemagnetized': None,    # None (defaults to author selection) or 'y'  
    'sample_count': None,      # None (defaults to author selection) or int: cutoff n (≥ x)
    'alpha_95': None,          # None (defaults to author selection) or float: cutoff A95 (≤ x degrees)
    'overprints': None,        # None (defaults to author selection) or 'y'  
    'remagnetizations': None,  # None (defaults to author selection) or 'y'
    'uncertain_struct': None,  # None (defaults to author selection) or 'y'
    'rotated': None,           # None (defaults to author selection) or 'y'
    'shallowed': None,         # None (defaults to author selection) or 'y' [***can also implement cutoff f-value here if desired***]
    'anomalous_dir': None,     # None (defaults to author selection) or float: cutoff distance (in degrees) between vgp and mean (≤ x degrees)
    'uncertain_age': None,     # None (defaults to author selection) or float: cutoff age resolution (in Myr) between min and max (≤ x Myr)
    'distinct_age': None,      # None (defaults to author selection) or 'y'
    'sub-time_units': None,    # None (defaults to author selection) or 'y'
    'rock_type': None,         # None (defaults to author selection) or string: 'all' or 'igneous' or 'sedimentary'
    'otherwise_rej': None,     # None (defaults to author selection) or 'y'
    }

Parse the inclusion criteria to numeric codes used in the vgp database

In [4]:
criteria_codes = []
if incl_criteria['author_selection'] == 1: pass #ignore all other criteria if original selection is to be used
elif incl_criteria['author_selection'] == 0:
    if incl_criteria['undemagnetized'] == 'y': criteria_codes.append(1)
    if incl_criteria['sample_count'] == type(int): criteria_codes.append(2)
    if incl_criteria['alpha_95'] == type(float) or incl_criteria['alpha_95'] == type(int): criteria_codes.append(3)
    if incl_criteria['overprints'] == 'y': criteria_codes.append(4)
    if incl_criteria['remagnetizations'] == 'y': criteria_codes.append(5)
    if incl_criteria['uncertain_struct'] == 'y': criteria_codes.append(6)
    if incl_criteria['rotated'] == 'y': criteria_codes.append(7)
    if incl_criteria['shallowed'] == 'y': criteria_codes.append(8)
    if incl_criteria['anomalous_dir'] == type(float) or incl_criteria['anomalous_dir'] == type(int): criteria_codes.append(9)
    if incl_criteria['uncertain_age'] == type(float) or incl_criteria['uncertain_age'] == type(int): criteria_codes.append(10)    
    if incl_criteria['distinct_age'] == 'y': criteria_codes.append(11)
    if incl_criteria['sub-time_units'] == 'y': criteria_codes.append(12)
    if incl_criteria['rock_type'] == 'all' or incl_criteria['rock_type'] == 'igneous' or incl_criteria['rock_type'] == 'sedimentary': criteria_codes.append(13)
    if incl_criteria['otherwise_rej'] == 'y': criteria_codes.append(14)
else:
    print ('invalid inclusion criterion selected for author_selection')
    
print ('the numeric codes selected: ', criteria_codes)

the numeric codes selected:  []


### Initialize new compilation dataframes and reference counters
We will append the data extracted / recalculated from each datasheet into a new compilation, and so need to initialize new dataframes.

In [5]:
df_vgp_compilation = pd.DataFrame()
df_pole_compilation = pd.DataFrame()

Several elements of the vgp database involve internal references between sites of a given datasheet, namely: those belonging to common rotated regions, a sequence of stratigraphically ordered sites, and sites with redundant data (e.g. 2 or more sites from synchronous units). In all cases, this referencing is achieved by way of simple numeric tags which are non-unique between datasheets. In order to preserve these references when we merge datasheets, we need to make each code unique. This is easily achieved with use of counters, which we also initialize here.

In [6]:
rot_cntr = 0    # counter for rotated area references
strat_cntr = 0  # counter for stratigraphic group references
synch_cntr = 0  # counter for synchronous unit references

## Example
In order to illustrate the workflow, we select an arbitrary single datasheet to process below, before executing the same process on the entire database.

In [7]:
files[['name']]

,name
0,test
1,test2
2,test3


In [8]:
file_idx = 2

### Split study- and site-level data
Each datasheet contains both study-level poles and site-level vgps. We split and assign these to separate dataframes and cast types for their constituent series.

In [9]:
def split_datasheet (files, file_idx): 
    df = pd.read_csv(files['path'][file_idx], skip_blank_lines = False, encoding = "ISO-8859-1") #, skip_blank_lines=True
    df_list = np.split(df, df[df.isnull().all(1)].index)
    df_poles = df_list[0]
    df_vgps = df_list[1].dropna(how='all')
    df_vgps = df_vgps.rename(columns = df_vgps.iloc[0]).drop(df_vgps.index[0]) # assign the first row as columns for the df_vgps
    
    #cast columns (using floats everywhere b/c columns with NaNs cannot be cast as ints...can always remove NaNs and recast as int later if needed)
    df_poles = df_poles.astype({'pole':float, 'name':str, 'slat':float, 'slon':float, 'N':float, 'dec':float, 'inc':float, 'k':float, 'alpha95':float, \
                                'f_corr': float, 'Plat':float, 'Plon':float, 'K':float, 'A95':float, 'dp':float, 'dm':float, 'mean_age':float, \
                                'min_age':float, '2sig_min':float, 'max_age':float, '2sig_max':float, 'uncer_dist':str, 'rock_typ_1':str, \
                                'rock_typ_2':str, 'rock_typ_3':str, 'R1':int, 'R2.1':float, 'R2.2':float, 'R3':float, 'R4':float, 'R5.1':float, \
                                'R5.2':float, 'R6':float, 'R7':float, 'pmag_ref':str, 'age_ref':str, 'pmag_comments':str, 'age_comments':str})
    
    df_vgps = df_vgps.astype({'name':str, 'fm./loc.':str, 'slat':float, 'slon':float, 'n':float, 'dec':float, 'inc':float, 'k':float, 'alpha95':float, \
                                'f_corr': float, 'VGP_lat':float, 'VGP_lon':float, 'K':float, 'A95':float, 'dp':float, 'dm':float, 'mean_age':float, \
                                'min_age':float, '2sig_min':float, 'max_age':float, '2sig_max':float, 'uncer_dist':str, 'rock_typ_1':str, \
                                'rock_typ_2':str, 'rock_typ_3':str, 'demag':float, 'struc_cont':float, 'rot_area':float, 'polarity':str, \
                                'strat_group':float, 'ordering':float, 'synch_unit':str, 'in_study_pole':str, 'rej_crit':str, 'pmag_ref':str, \
                                'age_ref':str, 'pmag_comments':str, 'age_comments':str})
    
    return (df_poles, df_vgps)

In [10]:
df_poles, df_vgps = split_datasheet(files, file_idx)

Have a look at the data

In [11]:
df_poles.head()

,pole,name,slat,slon,N,dec,inc,k,alpha95,f_corr,Plat,Plon,K,A95,dp,dm,mean_age,min_age,2sig_min,max_age,2sig_max,uncer_dist,rock_typ_1,rock_typ_2,rock_typ_3,R1,R2.1,R2.2,R3,R4,R5.1,R5.2,R6,R7,pmag_ref,age_ref,pmag_comments,age_comments,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,1.0,Quaternary age rocks; Western Central TMVB,NaN,NaN,10.0,2.9,37.5,38.0,7.9,NaN,86.7,314.0,43.0,7.5,NaN,NaN,NaN,0.00,NaN,2.58,NaN,uniform,igneous,volcanic,mafic to intermediate lavas,1,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0,Ruiz-Martínez et al. (2010),GTS2020,no field stability tests; structural coherence...,nan,NaN,NaN,NaN,NaN,NaN
1,2.0,late Miocene-Pliocene age rocks; Western Centr...,NaN,NaN,33.0,0.6,38.2,22.0,5.4,NaN,88.0,265.5,26.0,5.0,NaN,NaN,NaN,2.58,NaN,11.20,NaN,uniform,igneous,volcanic,mafic to intermediate lavas,1,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0,Ruiz-Martínez et al. (2010),GTS2020,no field stability tests; structural coherence...,min age is Pliocene-Quaternary boundary; max a...,NaN,NaN,NaN,NaN,NaN


In [12]:
df_vgps.head()

,name,fm./loc.,slat,slon,n,dec,inc,k,alpha95,f_corr,VGP_lat,VGP_lon,K,A95,dp,dm,mean_age,min_age,2sig_min,max_age,2sig_max,uncer_dist,rock_typ_1,rock_typ_2,rock_typ_3,demag,struc_cont,rot_area,polarity,strat_group,ordering,synch_unit,in_study_pole,rej_crit,pmag_ref,age_ref,pmag_comments,age_comments,NaN,NaN,NaN,NaN,NaN
4,mwnHIG,Western sector of TMVB,20.79,-105.48,11.0,343.6,24.6,37.0,6.9,NaN,72.4,140.4,NaN,NaN,NaN,NaN,10.20,9.40,NaN,11.00,NaN,normal,igneous,volcanic,mafic to intermediate lavas,NaN,NaN,NaN,nan,0.0,NaN,0,2,nan,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),nan,nan,NaN,NaN,NaN,NaN,NaN
5,mwrSF,Western sector of TMVB,20.89,-105.41,11.0,190.8,-45.3,2.0,30.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.10,10.90,NaN,11.30,NaN,normal,igneous,volcanic,mafic to intermediate lavas,NaN,NaN,NaN,nan,0.0,NaN,0,0,3,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),scattered directions,nan,NaN,NaN,NaN,NaN,NaN
6,pwnPLA,Western sector of TMVB,21.35,-105.24,8.0,351.1,31.3,60.0,6.4,NaN,80.5,138.4,NaN,NaN,NaN,NaN,NaN,2.58,NaN,3.50,NaN,uniform,igneous,volcanic,mafic to intermediate lavas,NaN,NaN,NaN,nan,0.0,NaN,0,2,nan,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),nan,nan,NaN,NaN,NaN,NaN,NaN
7,pwrLIB,Western sector of TMVB,21.58,-105.19,10.0,166.5,-41.1,492.0,2.2,NaN,77.4,176.4,NaN,NaN,NaN,NaN,NaN,2.58,NaN,3.50,NaN,uniform,igneous,volcanic,mafic to intermediate lavas,NaN,NaN,NaN,nan,0.0,NaN,0,2,nan,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),nan,nan,NaN,NaN,NaN,NaN,NaN
8,pwrJOL,Western sector of TMVB,21.40,-105.18,7.0,8.4,49.1,78.0,6.9,NaN,78.6,294.4,NaN,NaN,NaN,NaN,3.36,3.19,NaN,3.53,NaN,normal,igneous,volcanic,mafic to intermediate lavas,NaN,NaN,NaN,nan,0.0,NaN,0,2,nan,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),nan,nan,NaN,NaN,NaN,NaN,NaN


### Parse data
The datasheets mostly report only those data provided in the original publication, so some series will be empty. In some cases these include series which we later need to operate on, so we need to compute these missing series (where possible from other reported data) or dismiss these entries. For this we will utilize several functions from the auxiliary library.

Compute vgps, where absent.

In [13]:
df_vgps = aux.get_poles(df_vgps, 'name', 'slat', 'slon', 'dec', 'inc', 'VGP_lat', 'VGP_lon', file_idx, verbose=True)

Cross-check the reported vgps against the dec/inc and slat/slon. Where poles appear to have been inverted, flip back to the correct polarity. Flag any otherwise spurious vgps to be checked against the original report.

In [14]:
df_vgps = aux.xcheck_dirs_poles(df_vgps, 'name', 'slat', 'slon', 'dec', 'inc', 'VGP_lat', 'VGP_lon', file_idx, verbose=True)

vgp from site pwrLIB in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site pwnPAL in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site pwrJAL in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site pwrSJG in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site pwrFER in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site mcrARE in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site pcrCHA in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site mcrFIN in file index 2 appears to be inverted. Flipping back (but perhaps check original reference).
vgp from site pcrTRO in file index 2 appears to be inverted. Flipping back (but 

Determine the polarity of each entry and create a new series with the data cast into the same (reverse) polarity (which makes life a bit easier later).

In [15]:
df_vgps = aux.go_reverse(df_vgps, 'VGP_lat', 'VGP_lon', 'rev_VGP_lat', 'rev_VGP_lon', rev_mean=[0,-90])

Compute alpha 95s, where absent.

In [16]:
df_vgps = aux.get_alpha95s(df_vgps, 'name', 'n', 'alpha95', 'k', file_idx, verbose=True)

### Exchange local references for unique ones
Exchange the internal references for the rotation areas, strat groups and synchronous units of a given datasheet with database-wide unique IDs using the counters. This ensures the references remain uniquely identifiable after merger of the individual datasheets into a compilation.

In [17]:
def assign_uniq_ids (df, rot_cntr, strat_cntr, synch_cntr):  # exchange local references for unique codes
    
    # assign unique ids to rotated areas and stratigraphic groups
    df.rot_area.fillna(value='0')
    df['rot_area'] = df.apply(lambda row: row.rot_area + rot_cntr if not row.rot_area == 0 else row.rot_area, axis=1)
    df.strat_group.fillna(value='0')
    df['strat_group'] = df.apply(lambda row: row.strat_group + strat_cntr if not row.strat_group == 0 else row.strat_group, axis=1)
    
    # update counters with new max values from local lists
    rot_cntr = df['rot_area'].max()
    strat_cntr = df['strat_group'].max()

    # assign unique ids to synchronous units (note that some entries have an 'M' prefix that designates them as a local mean)
    df.synch_unit.fillna(value='0')
    df.synch_unit = df.synch_unit.astype('str') # ensure that synch_unit entries are strings
    df['synch_unit'] = df.apply(lambda row: ' '.join(re.findall("[a-zA-Z]+", row.synch_unit)) + str(int(''.join(filter(str.isdigit, row.synch_unit))) \
                                                    + synch_cntr) if not row.synch_unit == '0' else row.synch_unit, axis = 1)
    
    #update counter with new max value
    synch_cntr = pd.to_numeric(df['synch_unit'], 'coerce').max()

    return (df, rot_cntr, strat_cntr, synch_cntr)

In [18]:
df_vgps, rot_cntr, strat_cntr, synch_cntr = assign_uniq_ids(df_vgps, rot_cntr, strat_cntr, synch_cntr)

## Filter data
Now evaluate the entries against the specified inclusion criteria. We start by filtering any entries that don't have the right inclusion codes and those which fail any specified n, alpha95, age uncertainty and/or rock type criteria.

In [19]:
def init_filter (df, incl_criteria, criteria_codes):
    
    df['rej_crit'] = df['rej_crit'].fillna(0) # replace NaNs in this column with 0's
    
    # make a new temp. series to flag entries which do / don't pass the basic inclusion criteria according to rej_crit codes
    df['keep'] = df.apply(lambda row: True if row.in_study_pole != 0 or all(crit in criteria_codes for crit in [int(i) for i in str(row.rej_crit).split(',')]) \
                                else False, axis=1)
    
    # reject any entries with too small sample count
    if 2 in criteria_codes:
        df['keep'] = df.apply(lambda row: False if row.n < incl_criteria['sample_count'] else row.keep, axis=1)

    # reject any entries with too large alpha 95
    if 3 in criteria_codes:  
        df['keep'] = df.apply(lambda row: False if row.alpha95 > incl_criteria['alpha_95'] else row.keep, axis=1)

    # reject vgps with too large age uncertainty (as determined by diff b/w min and max)
    if 10 in criteria_codes:
        df['keep'] = df.apply(lambda row: False if (row.max_age - row.min_age) > incl_criteria['uncertain_age'] else row.keep, axis=1)

    # reject vgps with wrong rock type
    if 13 in criteria_codes: 
        if incl_criteria['rock_type'] == 'all': pass
        else: df['keep'] = df.apply(lambda row: False if row.rock_typ_1 != incl_criteria['rock_type'] else row.keep, axis=1)
            
    df.drop(df[df.keep == False].index, inplace=True)
    
    return df

In [20]:
df_vgps = init_filter(df_vgps, incl_criteria, criteria_codes)

We still need to find and reject any anomalous vgps (if such criteria were specified above). However, in order to evaluate this, we need a provisional paleopole. Before we compute that we need to remove any vgps with distinct ages (since they shouldn't contribute to the paleopole calculation). These temporally distinctive entries can be sent to the main compilation (as they have otherwise passed the basic inclusion criteria above) and deleted from the working dataframe here.

In [21]:
def strip_age_distinct (df, df_vgp_compilation, criteria_codes):

    # check if any entries with distinct age and flag with a NaN in the 'keep' column
    if 11 in criteria_codes:
        df['keep'] = df.apply(lambda row: np.nan if (row.keep == True and 11 in [int(i) for i in str(row.rej_crit).split(',')]) else row.keep, axis=1) 

        #pass these distinct age vgps to the vgp compilation
        df_distinct = df[df['keep'] == np.nan]
        df_vgp_compilation = pd.concat([df_vgp_compilation, df_distinct], axis=0)

        #drop the distinct vgps from the selected list
        df.drop(df_distinct.index, axis=0, inplace=True)
    
    return (df, df_vgp_compilation)

In [22]:
df_vgps, df_vgp_compilation = strip_age_distinct(df_vgps, df_vgp_compilation, criteria_codes)

## Compute provisional paleopole
Now we can compute a provisional paleopole and check for 'anomalous' vgps (as user-defined). To do this rigorously presents a potential recursive headache because we should first pre-average any synchronous units, but some of these could include vgps susequently recognized as 'anomalous', requiring calculation of a new provisional pole, etc. Here we instead adopt a simple approximation: defaulting to the subset of selected data which the original authors retained.

In [23]:
def strip_anomalous (df, criteria_codes):

    if 9 in criteria_codes: 
        
        #isolate the entries to be used for provisional paleopole calculation
        df_prov = df[(df['in_study_pole'] != 0) & (df['keep'] == True)]

        #calculate provisional paleopole
        ppole = ipmag.fisher_mean(dec = df_prov['rev_VGP_lon'].tolist(), inc = df_prov['rev_VGP_lat'].tolist())

        #identify anomalous vgps according to the specification above
        df['keep'] = df.apply(lambda row: False if (pmag.angle([row.rev_VGP_lon, row.rev_VGP_lat], [ppole['dec'], ppole['inc']]) \
                                                              > incl_criteria['anomalous_dir']) else row.keep, axis=1)

        #drop anomalous entries
        df.drop(df[df.keep == False].index, inplace=True)

    return df

In [24]:
df_vgps = strip_anomalous(df_vgps, criteria_codes)

### Remove sparse time_units collections
After filtering there may be some synchronous unit collections with too few entries to merit passing them onward as individual entries. In these cases, we can adopt any existing reported mean and discard the individual entries. Where there exists a sufficient number of individual entries we can pass them on and omit the reported mean.

In [25]:
def strip_sparse_time_units (df, incl_criteria):

    # first specify an n-specific cutoff value to decide whether to recalculate mean or retain the reported one.
    min_site_count = 3
    if incl_criteria['sample_count'] != None and incl_criteria['sample_count'].isdigit():
        if incl_criteria['sampl_count'] > min_site_count: min_site_count = int(incl_criteria['sampl_count'])
            
    # extract synchronous units from selected list and split into groups
    df['keep'] = df.apply(lambda row: False if row.synch_unit != '0' else row.keep, axis=1)
    df_redundant = df[df['synch_unit'] != '0']
    synch_units = df_redundant.groupby(['synch_unit'])
    
    # collect means into a list to make them easy to locate
    means = [x for x in synch_units.groups if x.isdigit() == False]

    # check the number of sites for each group
    for key, group in synch_units:
        if key.isdigit():    # ignore means
            
            if len(group) > min_site_count:   # if number of sites is sufficient, keep all individual sites
                df.loc[group.index.tolist(), 'keep'] = True

            elif ('M'+str(key)) in means:     # if number of sites is too low and mean is reported, keep mean
                mean_ent = synch_units.get_group('M'+str(key))
                df.loc[mean_ent.index.tolist(), 'keep'] = True
                df.loc[mean_ent.index.tolist(), 'synch_unit'] = '0' # set time_unit to 0 as there is now only 1 entry

            else:      # if number of sites is too low and no mean is reported, use site with smaller alpha95 (or higher n)
                df.loc[group['alpha95'].idxmin(), 'keep'] = True
                ### alternatively: group['n'].idxmax()
                df.loc[group['alpha95'].idxmin(), 'synch_unit'] = '0'  # set time_unit to 0 as there is now only 1 entry
                
    #drop discarded entries
    df.drop(df[df.keep == False].index, inplace=True)
    
    return df

In [26]:
df_vgps = strip_sparse_time_units(df_vgps, incl_criteria)

## Append filtered vgps to compilation
Pass the final selected entries to the vgp compilation.

In [27]:
df_vgp_compilation = pd.concat([df_vgp_compilation, df_vgps], axis=0)

## Final paleopole recalculation
Finally, recalculate the paleopole based on only the filtered set of vgp data, and after pre-averaging any synchronous units.

In [28]:
def average_synch_units (df): # pre-average any synchronous units
    
    df['keep'] = df.apply(lambda row: False if row.synch_unit != '0' else row.keep, axis=1)
    df_redundant = df[df['synch_unit'] != '0']
    synch_units = df_redundant.groupby(['synch_unit'])
    
    for key, group in synch_units:
        mean_age = group['mean_age'].mean(axis=0) # get mean age from among time_unit sites *** NOTE THIS DOESN'T COLLECT / PASS ON UNCERTAINTIES ***
        
        vgp = ipmag.fisher_mean(dec = group['rev_VGP_lon'].tolist(), inc = group['rev_VGP_lat'].tolist()) # compute mean vgp from among time_units
        
        df.append({'VGP_lon': vgp['dec'], 'rev_VGP_lon': vgp['dec'], 'VGP_lat': vgp['inc'], 'rev_VGP_lat': vgp['inc'],
                   'A95': vgp['alpha95'], 'mean_age': mean_age, 'keep': True}, ignore_index=True)    # other entries could be added but aren't presently needed
        
    #drop discarded entries
    df.drop(df[df.keep == False].index, inplace=True)
    
    return df    

In [29]:
df_vgps = average_synch_units(df_vgps)

Now compute the final paleopole, determine its corresponding age, and append it to the re-calculated paleopole dataframe.

In [30]:
def get_pole_data (df):

    pole = ipmag.fisher_mean(dec = df['rev_VGP_lon'].tolist(), inc = df['rev_VGP_lat'].tolist())
    mean_site = ipmag.fisher_mean(dec = df['slon'].tolist(), inc = df['slat'].tolist())
    
    mean_age = df['mean_age'].mean(axis=0) # get mean pole age  *** NOTE THIS DOESN'T COLLECT / PASS ON UNCERTAINTIES ***
    
    recomputed_pole_data = {'slat': mean_site['inc'], 'slon': mean_site['dec'], 'N': pole['n'], 'Plat': pole['inc'], 'Plon': pole['dec'], \
                 'K': pole['k'], 'A95': pole['alpha95'], 'mean_age': mean_age}

    return recomputed_pole_data

In [31]:
df_pole_compilation = pd.DataFrame(data=None, columns=df_poles.columns) # re-initialized in order to copy over column names
recomputed_pole_data = get_pole_data(df_vgps)

# Execute on entire database
Having demonstrated the workflow on an example datasheet, we now execute it on the entire dataset, cycling through all the datasheets.

In [32]:
# reset initializations
df_vgp_compilation = pd.DataFrame()
df_pole_compilation = pd.DataFrame()
rot_cntr = 0
strat_cntr = 0
synch_cntr = 0

for i in files.index:   # cycle over each file in database
    
    # import data and assign to dataframes
    df_poles, df_vgps = split_datasheet(files, i)
    
    # parse data
    df_vgps = aux.get_poles(df_vgps, 'name', 'slat', 'slon', 'dec', 'inc', 'VGP_lat', 'VGP_lon', file_idx, verbose=True)
    df_vgps = aux.xcheck_dirs_poles(df_vgps, 'name', 'slat', 'slon', 'dec', 'inc', 'VGP_lat', 'VGP_lon', file_idx, verbose=True)
    df_vgps = aux.go_reverse(df_vgps, 'VGP_lat', 'VGP_lon', 'rev_VGP_lat', 'rev_VGP_lon', rev_mean=[0,-90])
    df_vgps = aux.get_alpha95s(df_vgps, 'name', 'n', 'alpha95', 'k', file_idx, verbose=True)

    # exchange local references for unique ids
    df_vgps, rot_cntr, strat_cntr, synch_cntr = assign_uniq_ids(df_vgps, rot_cntr, strat_cntr, synch_cntr)
    
    # filter data
    df_vgps = init_filter(df_vgps, incl_criteria, criteria_codes)
    df_vgps, df_vgp_compilation = strip_age_distinct(df_vgps, df_vgp_compilation, criteria_codes)
    df_vgps = strip_anomalous(df_vgps, criteria_codes)
    df_vgps = strip_sparse_time_units(df_vgps, incl_criteria)

    # append filtered vgps to compilation
    df_vgp_compilation = pd.concat([df_vgp_compilation, df_vgps], axis=0)
    
    # re-calculate study-level pole
    df_vgps = average_synch_units(df_vgps)
    recomputed_pole_data = get_pole_data(df_vgps)
    
    # append new pole entry to compilation
    if i == 0: df_pole_compilation = pd.DataFrame(data=None, columns=df_poles.columns)
    df_pole_compilation = df_pole_compilation.append(recomputed_pole_data, ignore_index=True)
    
# write out compilation
out_dir = data_path_vgp + '/compilations/'
df_vgp_compilation.to_csv(out_dir + 'vgps.csv', index=False)
df_pole_compilation.to_csv(out_dir + 'poles.csv', index=False)

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'